# भाग 12: एनक्रिप्टेड डेटा पर एनक्रिप्टेड एनएन को प्रशिक्षित करें (Train an Encrypted NN on Encrypted Data)

इस नोटबुक में, हम उन सभी तकनीकों का उपयोग करने जा रहे हैं जो हमने अब तक सीखा है न्यूरल नेटवर्क प्रशिक्षण (और भविष्यवाणी) के लिए जबकि मॉडल और डेटा दोनों एन्क्रिप्टेड हो।

विशेष रूप से, हम अपने कस्टम Autograd इंजन को प्रस्तुत करते हैं जो एन्क्रिप्टेड संगणना पर काम करता है।

लेखक:
- Andrew Trask - Twitter: [@iamtrask](https://twitter.com/iamtrask)
- Jason Paumier - Github: [@Jasopaum](https://github.com/Jasopaum)
- Théo Ryffel - Twitter: [@theoryffel](https://twitter.com/theoryffel)

अनुवादक:
- Yugandhar Tripathi - Github: [@Yugandhartripathi](https://github.com/Yugandhartripathi)

# चरण 1: श्रमिक और खिलौना डेटा बनाएँ

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import syft as sy

In [ ]:
# Set everything up
hook = sy.TorchHook(torch) 

alice = sy.VirtualWorker(id="alice", hook=hook)
bob = sy.VirtualWorker(id="bob", hook=hook)
james = sy.VirtualWorker(id="james", hook=hook)

In [ ]:
# A Toy Dataset
data = torch.tensor([[0,0],[0,1],[1,0],[1,1.]])
target = torch.tensor([[0],[0],[1],[1.]])

# A Toy Model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2, 2)
        self.fc2 = nn.Linear(2, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x
model = Net()

# चरण 2: मॉडल और डेटा को एन्क्रिप्ट करें

यहाँ एन्क्रिप्शन दो चरणों में आता है। चूँकि सिक्योर मल्टी पार्टी कंपटीशन केवल पूर्णांकों पर काम करता है, दशमलव अंकों के साथ संख्याओं को संचालित करने के लिए (जैसे कि वज़न और सक्रियण), हमें अपने सभी नंबरों को निश्चित परिशुद्धता का उपयोग करके सांकेतिक शब्दों में बदलना चाहिए, जो हमें दशमलव परिशुद्धता के कई बिट्स देगा। हम इसे कॉल कर रहे हैं `.fix_precision()`।

इसके बाद हम अन्य डेमो की तरह `.share()` कॉल कर सकते हैं, जो ऐलिस और बॉब के बीच साझा करके सभी मूल्यों को एन्क्रिप्ट करेगा। ध्यान दें कि हमने `requires_grad` को `True` भी सेट किया है, जो एन्क्रिप्टेड डेटा के लिए एक विशेष `auto_grad` विधि भी जोड़ता है। दरअसल, चूँकि सिक्योर मल्टी-पार्टी कम्प्यूटेशन फ्लोट वैल्यू पर काम नहीं करता है, इसलिए हम सामान्य PyTorch autograd का उपयोग नहीं कर सकते हैं। इसलिए, हमें एक विशेष `AutogradTensor` नोड को जोड़ने की आवश्यकता है जो backpropagation के लिए ढाल ग्राफ की गणना करता है। आप किसी भी तत्व को यह देखने के लिए प्रिंट कर सकते हैं कि इसमें `AutogradTensor` शामिल है।

In [ ]:
# We encode everything
data = data.fix_precision().share(bob, alice, crypto_provider=james, requires_grad=True)
target = target.fix_precision().share(bob, alice, crypto_provider=james, requires_grad=True)
model = model.fix_precision().share(bob, alice, crypto_provider=james, requires_grad=True)

In [ ]:
print(data)

# चरण 3: ट्रेन

और अब हम साधारण टेंसर लॉजिक का उपयोग करके प्रशिक्षण ले सकते हैं।

In [ ]:
opt = optim.SGD(params=model.parameters(),lr=0.1).fix_precision()

for iter in range(20):
    # 1) erase previous gradients (if they exist)
    opt.zero_grad()

    # 2) make a prediction
    pred = model(data)

    # 3) calculate how much we missed
    loss = ((pred - target)**2).sum()

    # 4) figure out which weights caused us to miss
    loss.backward()

    # 5) change those weights
    opt.step()

    # 6) print our progress
    print(loss.get().float_precision())

नुकसान वास्तव में कम हो गया!

## निश्चित परिशुद्धता का प्रभाव
आप आश्चर्यचकित हो सकते हैं कि कैसे सब कुछ एन्क्रिप्ट करने से घटती क्षति प्रभावित होती है। दरअसल, क्योंकि सैद्धांतिक गणना समान है, संख्या गैर-एन्क्रिप्टेड प्रशिक्षण के बहुत करीब है। आप एन्क्रिप्शन के बिना एक ही उदाहरण चलाकर और मॉडल के एक नियतात्मक प्रारंभिककरण के साथ इस तरह का सत्यापन कर सकते हैं जैसे कि मॉडल में `__init__ `:
```
with torch.no_grad():
    self.fc1.weight.set_(torch.tensor([[ 0.0738, -0.2109],[-0.1579,  0.3174]], requires_grad=True))
    self.fc1.bias.set_(torch.tensor([0.,0.1], requires_grad=True))
    self.fc2.weight.set_(torch.tensor([[-0.5368,  0.7050]], requires_grad=True))
    self.fc2.bias.set_(torch.tensor([-0.0343], requires_grad=True))
```

निश्चित अंतर में परिवर्तित होते समय प्रदर्शन किए गए मानों के गोलाई के कारण आप जो अंतर देख सकते हैं, वह है। डिफ़ॉल्ट `सटीक_ आंशिक` 3 है और यदि आप इसे 2 से नीचे लाते हैं, तो स्पष्ट पाठ प्रशिक्षण बढ़ जाता है, जबकि यह कम कर देता है यदि आप` परिशुद्धता_फैक्शनल = 4` चुनते हैं।

# बधाई हो!!! - समुदाय में शामिल होने का समय!

इस नोटबुक ट्यूटोरियल को पूरा करने पर बधाई! यदि आपने इसका आनंद लिया और एआई और एआई आपूर्ति श्रृंखला (डेटा) के विकेन्द्रीकृत स्वामित्व के संरक्षण, गोपनीयता की ओर आंदोलन में शामिल होना चाहते हैं, तो आप निम्न तरीकों से ऐसा कर सकते हैं!

### PySyft को Github पर Star करें!

हमारे समुदाय की मदद करने का सबसे आसान तरीका रिपॉजिटरी को अभिनीत करना है! यह हमारे द्वारा बनाए जा रहे कूल टूल्स के बारे में जागरूकता बढ़ाने में मदद करता है।

- [स्टार PySyft](https://github.com/OpenMined/PySyft)

### हमारे Slack में शामिल हों!

नवीनतम प्रगति पर अद्यतित रहने का सबसे अच्छा तरीका हमारे समुदाय में शामिल होना है!

- [slack.openmined.org से जुड़ें](http://slack.openmined.org)

### एक कोड परियोजना में शामिल हों!

हमारे समुदाय में योगदान करने का सबसे अच्छा तरीका एक कोड योगदानकर्ता बनना है! किसी भी समय आप PySyft Github जारी करने वाले पृष्ठ पर जा सकते हैं और "प्रोजेक्ट्स" के लिए फ़िल्टर कर सकते हैं। यह आपको सभी शीर्ष स्तर के टिकट दिखाएगा कि आप किन परियोजनाओं में शामिल हो सकते हैं! यदि आप किसी प्रोजेक्ट से जुड़ना नहीं चाहते हैं, लेकिन आप थोड़ी सी कोडिंग करना चाहते हैं, तो आप "अच्छा पहला अंक" चिह्नित गिथब मुद्दों की खोज करके अधिक "वन ऑफ" मिनी-प्रोजेक्ट्स की तलाश कर सकते हैं।

- [PySyft Projects](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3AProject)
- [अच्छा पहला अंक टिकट(Good First Issue Tickets)](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3A%22good+first+fueue%22)

### दान करना

यदि आपके पास हमारे कोडबेस में योगदान करने का समय नहीं है, लेकिन फिर भी समर्थन उधार देना चाहते हैं, तो आप हमारे ओपन कलेक्टिव में भी एक बैकर बन सकते हैं। सभी दान हमारी वेब होस्टिंग और अन्य सामुदायिक खर्चों जैसे कि हैकाथॉन और मीटअप की ओर जाते हैं!

[OpenMined का ओपन कलेक्टिव पेज](https://opencollective.com/openmined)